In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000212DBDF8090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000212DD413250>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Devansh and I am a Data Scientist")])

AIMessage(content="Hi Devansh, it's nice to meet you!\n\nAs a Data Scientist, what are you currently working on? Do you have any interesting projects you'd like to talk about?  \n\nI'm always interested in learning more about the work data scientists do.  Perhaps I can even help with some tasks or provide information related to your projects.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 22, 'total_tokens': 98, 'completion_time': 0.138181818, 'prompt_time': 0.002149688, 'queue_time': 0.24142572599999998, 'total_time': 0.140331506}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3e75be1-8d1d-4868-baa8-cba39d1a8f7a-0', usage_metadata={'input_tokens': 22, 'output_tokens': 76, 'total_tokens': 98})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hi my name is Devansh and i am a Data Scientist"),
        AIMessage(content="Hi Devansh, it's nice to meet you!\n\nAs a Data Scientist, what are you currently working on? Do you have any interesting projects you'd like to talk about?  \n\nI'm always interested in learning more about the work data scientists do.  Perhaps I can even help with some tasks or provide information related to your projects.\n"),
        HumanMessage(content="Hey, what is my name and what do i do?")
        
    ]
)

AIMessage(content="You said your name is Devansh, and you are a Data Scientist!  😊 \n\nIs there anything else you'd like to tell me about yourself or your work?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 117, 'total_tokens': 156, 'completion_time': 0.070909091, 'prompt_time': 0.010180398, 'queue_time': 0.343309592, 'total_time': 0.081089489}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--35215777-1ff9-4747-8158-464e6173a45a-0', usage_metadata={'input_tokens': 117, 'output_tokens': 39, 'total_tokens': 156})

In [6]:
## message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # whatever chat is happening it should be stored in the current session id
    return store[session_id]
with_message_hsitory = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [9]:
response = with_message_hsitory.invoke(
    [HumanMessage(content='Hi I am Devansh and i am a Data Sientist')],
    config=config
)

In [10]:
response.content

"Hey Devansh, that's awesome! \n\nWhat kind of data science work are you into?  Do you specialize in a particular area like machine learning, deep learning, or data visualization?  \n\nI'm always eager to learn more about what people are doing in the field. 😊 \n\n"

In [11]:
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_hsitory.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

"As an AI, I have no access to your personal information, including your name. If you'd like to tell me your name, I'd be happy to know!\n"

#### Prompt Templates

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Devansh")]})

AIMessage(content="Hello Devansh, it's nice to meet you! \n\nI'm happy to help with any questions you have. What can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 32, 'total_tokens': 71, 'completion_time': 0.070909091, 'prompt_time': 0.002319314, 'queue_time': 0.24861039800000004, 'total_time': 0.073228405}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--229fbf88-b1c7-496e-bf0c-063be953c6d6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 39, 'total_tokens': 71})

In [17]:
with_message_hsitory = RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_hsitory.invoke(
    [HumanMessage(content="Hi my name is Devansh")],
    config = config
)

In [19]:
response.content

"Hi Devansh, it's nice to meet you!\n\nWhat can I do for you today? 😊  \n\n"

In [20]:
## adding complexity
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt|model

In [21]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Devansh")],"language":"Spanish"})
response.content

'¡Hola Devansh! \n\nMucho gusto en conocerte. ¿Cómo puedo ayudarte hoy? \n'

In [22]:
with_message_hsitory = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [23]:
config = {"configurable":{"session_id":"chat4"}}
resp = with_message_hsitory.invoke(
    {"messages":[HumanMessage(content="Hi my name is Devansh")],"language":"French"},
    config=config
)
resp.content

"Bonjour Devansh !\n\nEnchanté de te rencontrer. Comment puis-je t'aider aujourd'hui ? \n"

In [24]:
resp = with_message_hsitory.invoke(
    {"messages":[HumanMessage(content="What is my name")],"language":"French"},
    config=config
)
resp.content

'Ton nom est Devansh.  \n'

#### Managing the conversation history

In [31]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial = False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good AI asistant"),
    HumanMessage(content="Hi! I am Devansh"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like dark chocolates"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes! Bring it on")
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good AI asistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes! Bring it on', additional_kwargs={}, response_metadata={})]

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain= (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
response = chain.invoke({"messages":messages + [HumanMessage(content="What ice cream do i like?")],
              "language":"English"})
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor. \n\nWhat's your favorite flavor?  🍦😋\n"

In [33]:

response = chain.invoke({"messages":messages + [HumanMessage(content="What math problem did i ask")],
              "language":"English"})
response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,get_session_history,input_messages_key="messages",
)
config = {"configurable":{"session_id":"chat5"}}

response = chain.invoke({"messages":messages + [HumanMessage(content="What problem did i ask")],
              "language":"English"},
              config=config,)
response.content

"You haven't asked me any problems yet! 😄 \n\nWhat can I help you with?\n"